In [2]:
library(tidyverse)
library(repr)
library(tidymodels)
options(repr.matrix.max.rows = 6)
source('tests.R')
source('cleanup.R')

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.6     ✔ purrr   0.3.4
✔ tibble  3.1.7     ✔ dplyr   1.0.9
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.2     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

── Attaching packages ────────────────────────────────────── tidymodels 1.0.0 ──

✔ broom        1.0.0     ✔ rsample      1.0.0
✔ dials        1.0.0     ✔ tune         1.0.0
✔ infer        1.0.2     ✔ workflows    1.0.0
✔ modeldata    1.0.0     ✔ workflowsets 1.0.0
✔ parsnip      1.0.0     ✔ yardstick    1.0.0
✔ recipes      1.0.1     

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ scales::discard() masks purrr::discard()
✖ dplyr::filter()   masks stats::filter()
✖ recipes::fixed()  masks stringr::fixed()
✖ dplyr::lag()      masks stats::lag()
✖ yardstick::spec() masks r

ERROR: Error in file(filename, "r", encoding = encoding): cannot open the connection


In [18]:
# I'm just providing a few ways of organizing the data...


titles_data <- read_csv("data/titles.csv")

titles_movie_data <- titles_data %>%
    filter(release_year == 2022, type == "MOVIE") %>%
    select(-description, -seasons, -type, -id, -release_year)

titles_show_data <- titles_data %>%
    filter(release_year == 2022, type == "SHOW") %>%
    select(-description, -type, -id, -release_year)
    
titles_movie_data
titles_show_data

Rows: 5850 Columns: 15
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (8): id, title, type, description, age_certification, genres, production...
dbl (7): release_year, runtime, seasons, imdb_score, imdb_votes, tmdb_popula...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


title,age_certification,runtime,genres,production_countries,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score
<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
The Gray Man,PG-13,128,"['thriller', 'action']","['US', 'CZ']",tt1649418,6.6,46071,510.268,6.851
RRR,R,187,"['drama', 'action']",['IN'],tt8178634,8.0,87515,53.526,7.800
The Man from Toronto,PG-13,110,"['action', 'thriller', 'drama', 'romance', 'comedy']",['US'],tt11671006,5.8,31497,1439.906,6.449
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
Celeb Five: Behind the Curtain,NA,55,"['comedy', 'documentation']",['KR'],tt18570468,4.7,29,1.674,5
Kongsi Raya,NA,102,['comedy'],['MY'],tt16806990,7.0,66,2.112,NA
Sun of the Soil,NA,26,[],[],NA,NA,NA,1.179,7


title,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score
<chr>,<chr>,<dbl>,<chr>,<chr>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
Resident Evil,TV-MA,51,"['scifi', 'thriller', 'action', 'horror']",['US'],1,tt9660182,3.7,24982,1387.392,5.956
The Lincoln Lawyer,TV-MA,50,"['crime', 'drama', 'thriller']",['US'],1,tt13833978,7.7,30295,68.925,8.100
Man vs. Bee,TV-PG,11,"['comedy', 'family']",['GB'],1,tt13640670,6.8,22287,260.169,7.477
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
Byron Baes,TV-MA,39,['reality'],['AU'],1,tt14537060,2.6,498,1.658,6
African Knock Out Show,TV-14,32,[],['US'],1,NA,NA,NA,0.839,NA
Only Jokes Allowed,TV-MA,16,['comedy'],['ZA'],1,tt17052148,4.9,45,1.629,10


I'm thinking we could use regression to predict imdb score based on imdb votes, tmdb popularity and tmdb score. Then for our visuals, we can just plot each of those variables against imdb score so a plot for each of the following: imdb score vs imdb votes, imdb score and tmdb popularity, imdb score and tmdb score. 

In [ ]:
source('cleanup.R')